# Day 1：数据接入与可视化（WSL2 + venv 环境示例）

目标：
- 给出在 WSL2 中创建并激活 Python 虚拟环境的命令示例；
- 用 AkShare 下载示例股票（日线），并保存为 parquet 与 CSV；
- 绘制收盘价与成交量（以及可选的 K 线图）；
- 做简单的数据质量检查（空值、复权列检查、时间顺序、重复日期）。

说明：在继续之前建议在 WSL2 的 Linux 文件系统中建立项目目录（例如 `~/quant_project`），并在该目录创建虚拟环境。

## 在 WSL2 中创建并激活 venv（示例命令）
在 WSL2 终端中执行：
```bash
mkdir -p ~/quant_project && cd ~/quant_project
python3 -m venv venv
source venv/bin/activate
python -m pip install --upgrade pip setuptools wheel
```
如果你更倾向于 conda，也可以使用 Miniconda/conda 环境。

## 安装快速依赖（在激活的 venv 下运行）
下面是一个最小依赖集合，适合 Day1：
```bash
!pip install --upgrade pip
!pip install akshare pandas pyarrow matplotlib mplfinance jupyterlab
```
（注：在 notebook 中可直接执行带 `!` 的 pip 安装命令；生产/长期建议把依赖写入 requirements.txt 并在 venv 外部用 pip 安装。）

In [ ]:
# 环境与库检查（在激活 venv 后运行）
import sys
import platform
import pandas as pd
import numpy as np
import akshare as ak
import matplotlib
import matplotlib.pyplot as plt
print('python', sys.version)
print('platform', platform.platform())
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('akshare', ak.__version__)
print('matplotlib', matplotlib.__version__)


## 用 AkShare 下载示例股票日线并保存（示例：中国银行 sh601988）
说明：AkShare 的 `stock_zh_a_daily` 支持 `sh`/`sz` 前缀，例如 `sh601988`, `sz000001`。

In [ ]:
import os
import fastparquet
# os.makedirs('data', exist_ok=True)

# 示例 symbol，可以替换为你想要的股票代码
symbol = 'sh601988'  # 中国银行示例；改成 'sz000001' 等

# 拉取日线数据
df = ak.stock_zh_a_daily(symbol=symbol)

# 确保有日期列并统一列名（akshare 返回通常包含 'date','open','high','low','close','volume'）
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
else:
    # 如果 index 是日期（少见），重置 index
    df = df.reset_index()

df = df.rename(columns={
    'open':'open', 'high':'high', 'low':'low', 'close':'close', 'volume':'volume'
})

print('download rows:', len(df))
print(df.head())

# 保存为 parquet 与 csv
parquet_path = f'../data/{symbol.replace("/","_")}.parquet'
csv_path = f'../data/{symbol.replace("/","_")}.csv'
df.to_parquet(parquet_path, index=False,engine='fastparquet')
df.to_csv(csv_path, index=False)
print('Saved:', parquet_path, csv_path)


## 简单数据质量检查
我们检查：空值、重复日期、是否有复权因子（例如 `adj_factor`）、时间是否升序。

In [ ]:
# 空值统计
print('null counts:')
print(df.isnull().sum())

# 重复日期
if 'date' in df.columns:
    dup = df['date'].duplicated().sum()
    print('duplicate dates:', dup)

# 时间顺序（是否升序）
if 'date' in df.columns:
    is_sorted = df['date'].is_monotonic_increasing
    print('date is_monotonic_increasing:', is_sorted)
    if not is_sorted:
        df = df.sort_values('date').reset_index(drop=True)

# 复权因子检测（若存在则输出示例）
possible_adj_cols = [c for c in df.columns if 'adj' in c.lower() or '复权' in c]
print('possible adj columns:', possible_adj_cols)
if 'adj_factor' in df.columns:
    print('adj_factor sample:')
    print(df[['date','adj_factor']].tail())
else:
    print('没有发现标准名称的复权因子列（如 adj_factor）。若要用复权价格，请用 Tushare 获取或用复权因子手动计算）。')


## 绘制收盘价与成交量（matplotlib）

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('classic')

fig, axes = plt.subplots(2, 1, sharex=True, figsize=(12, 8), gridspec_kw={'height_ratios':[3,1]})
axes[0].plot(df['date'], df['close'], label='Close', color='tab:blue')
axes[0].set_title(f"{symbol} Close Price")
axes[0].legend()
axes[1].bar(df['date'], df['volume'], color='tab:gray')
axes[1].set_title('Volume')
plt.xlabel('Date')
plt.tight_layout()
plt.savefig('../data/stock_price_volume.png', dpi=300, bbox_inches='tight')
plt.show()



## 可选：绘制 K 线图（mplfinance）
如果已安装 `mplfinance`，下面的代码会画出带成交量的 K 线图。请确保 `date` 列已是 Datetime 且为索引，且列名匹配 `Open/High/Low/Close/Volume`。

In [ ]:
try:
    import mplfinance as mpf
    df_k = df.copy()
    df_k['date'] = pd.to_datetime(df_k['date'])
    df_k = df_k.set_index('date')
    # 将列名改为 mplfinance 期望的格式
    rename_map = {}
    for c in ['open','high','low','close','volume']:
        if c in df_k.columns:
            rename_map[c] = c.capitalize() if c != 'volume' else 'Volume'
    df_k = df_k.rename(columns=rename_map)
    # 只保留必要列（若缺列会抛错）
    required_cols = ['Open','High','Low','Close','Volume']
    if all([col in df_k.columns for col in required_cols]):
        mpf.plot(df_k[required_cols], type='candle', volume=True, mav=(5,10,20), figsize=(12,8),savefig='../data/stock_kline.png')
    else:
        print('K线图需要 Open/High/Low/Close/Volume 列，当前数据列为：', df_k.columns.tolist())
except Exception as e:
    print('绘制 K 线图失败（未安装 mplfinance 或数据列不完整）。错误：', e)
    print('可通过 pip install mplfinance 来安装。')


## 小结与下一步
- 你应该在 WSL2 中的项目目录下运行 jupyter lab 或 jupyter notebook：
  ```bash
  jupyter lab --no-browser --ip=0.0.0.0
  ```
  然后在 Windows 浏览器中打开 http://localhost:8888 访问并打开本 notebook。
- 本笔记下载的数据会保存到 `data/` 目录（parquet 与 csv），后续 Day2/Day3 可以直接读取 parquet 做回测与特征工程。
- 其实我是在windows下实现上述代码的，我的wsl崩了 :(\
  你可以和我一样使用
  ```
  python -m jupyter lab
  ```